In [1]:
import os
import time
import json

import concurrent
import numpy as np
import pandas as pd
from tqdm import tqdm
from evaluation_functions import evaluate
from openai_functions import extract_ade_terms

## Set Up

### Variables

In [41]:
# train
# drug_file = 'data/train_drug_label_text.csv'
# manual_file = 'data/train_drug_label_text_manual_ades.csv'

# test
drug_file = 'data/test_drug_label_text.csv'
manual_file = 'data/test_drug_label_text_manual_ades.csv'

# my_max = 10000

In [42]:
drugs = pd.read_csv(drug_file)
manual_ades = pd.read_csv(manual_file)
set_type = drug_file.split('/')[1].split('_')[0] # assuming file follows format "train_..." or "test...."

all_sections = drugs.query("section_name != 'all-concat'").groupby('drug_name')['section_text'].apply(' '.join).reset_index()
all_sections.insert(1, "section_name", ["all-concat" for _ in range(all_sections.shape[0])])
drugs = pd.concat([drugs, all_sections])

In [43]:
drugs.shape

(336, 3)

## Run GPT

In [44]:
outputs = {}

In [45]:
config = json.load(open('./config.json'))

organization = ""

api_source = 'OpenAI'

api_key = config[api_source]['openai_api_key'] #constants.AZURE_OPENAI_KEY
api_endpoint = config[api_source]['openai_api_endpoint'] 

gpt_model = config[api_source]["gpt_model"]
# gpt_model = "gpt-4-turbo-preview"
gpt_model = "gpt-3.5-turbo-0125"

temperature = 0

In [46]:
nruns = 3

system_options = {
    "no-system-prompt": "",
    "pharmexpert-v0": "You are an expert in pharmacology.",
    "pharmexpert-v1": "You are an expert in medical natural language processing, adverse drug reactions, pharmacology, and clinical trials."
}

prompt_options = {
    "fatal-prompt-v2": """
Extract all adverse reactions as they appear, including all synonyms.
mentioned in the text and provide them as a comma-separated list.
If a fatal event is listed add 'death' to the list.
The text is :'{}' 
""",
    "fatal-prompt-v3": """
Extract all adverse reactions as they appear, including all synonyms.
mentioned in the text and provide them as a comma-separated list. If a
negated adverse reaction appears extract it and include a <negated> tag.
If a fatal event is listed add 'death' to the list.
The text is :'{}' 
"""
}

system_name = "pharmexpert-v1"
system_content = system_options[system_name]

prompt_name = "fatal-prompt-v2"
prompt = prompt_options[prompt_name]

gpt_params = [f"temp{temperature}"]

output_file_basename = '{}_{}_{}_{}_{}_{}'.format(api_source, gpt_model, prompt_name, system_name, '-'.join(gpt_params), set_type)
output_file_basename

'OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test'

In [47]:
# run GPT
for i in range(nruns):
    run_key = "{}_run{}".format(output_file_basename, i)
    print(run_key)
    if run_key in outputs:
        print(f"Run {run_key} already started will pick up from where it was left off.")
    elif os.path.exists('results/{}.csv'.format(run_key)):
        gpt_output = pd.read_csv('results/{}.csv'.format(run_key))
        outputs[run_key] = gpt_output
        print(f"Run {run_key} started, loading from disk and pick up from where it was left off.")
    
    start = time.time()
    results = list()
    rows_to_run = list()
    for _, row in drugs.iterrows():
        name, section = row['drug_name'], row['section_name']

        if run_key in outputs:
            prev_run_results = outputs[run_key].query(f"drug_name == '{name}'").query(f"section_name == '{section}'")
            if prev_run_results.shape[0]==1:
                results.append([name, section, prev_run_results.gpt_output.values[0]])
            else:
                rows_to_run.append(row)
        else:
            rows_to_run.append(row)
        
    print(f"Loaded {len(results)} rows from file since they were already run.")
    print(f"There remains {len(rows_to_run)} rows to run.")

    def run_iteration(row):
        name, section = row['drug_name'], row['section_name']
        text = row['section_text']
        try:
            gpt_out = extract_ade_terms(api_source, config, gpt_model, system_content, prompt, text, temperature)
            return [name, section, gpt_out]
        except Exception as err:
            print(f"Encountered an exception for row: {name} {section}. Error message below:")
            print(err)
            return None
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as exec:
        results.extend(list(tqdm(
            exec.map(run_iteration, rows_to_run), 
            total=len(rows_to_run)
        )))
            
    gpt_output = pd.DataFrame(
        [r for r in results if r is not None],
        columns=['drug_name', 'section_name', 'gpt_output']
    )
    end = time.time()
    
    if gpt_output.shape[0] > 0:
        outputs[run_key] = gpt_output
        gpt_output.to_csv('results/{}.csv'.format(run_key))
    
    print(f"Run: {run_key}, time elapsed: {end-start}s.")

OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run0
Loaded 0 rows from file since they were already run.
There remains 336 rows to run.


 27%|██▋       | 92/336 [00:45<01:22,  2.95it/s] 

Encountered an exception for row: LATUDA all-concat. Error message below:
Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 17261 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


100%|██████████| 336/336 [11:26<00:00,  2.04s/it] 


Run: OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run0, time elapsed: 686.5511450767517s.
OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run1
Loaded 0 rows from file since they were already run.
There remains 336 rows to run.


...

Run: OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run1, time elapsed: 169.22306394577026s.
OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run2
Loaded 0 rows from file since they were already run.
There remains 336 rows to run.


 53%|█████▎    | 179/336 [01:32<01:21,  1.92it/s]

Encountered an exception for row: LATUDA all-concat. Error message below:
Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 17261 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


100%|██████████| 336/336 [03:34<00:00,  1.57it/s]

Run: OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run2, time elapsed: 214.4730348587036s.


## Evaluation

In [48]:
for run_key in sorted(outputs.keys()):
    print(run_key)

OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run0
OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run1
OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run2


In [49]:
evaluate(outputs, manual_ades, 'strict')
evaluate(outputs, manual_ades, 'lenient')

Running strict evaluation and saving results to disk.
OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run0


100%|██████████| 99/99 [00:02<00:00, 40.11it/s]


OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run1


100%|██████████| 99/99 [00:02<00:00, 40.99it/s]


OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run2


...

100%|██████████| 99/99 [00:12<00:00,  7.98it/s]


OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run1


100%|██████████| 99/99 [00:12<00:00,  7.97it/s]


OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v1_temp0_test_run2


100%|██████████| 99/99 [00:12<00:00,  7.85it/s]
